<a href="https://colab.research.google.com/github/mietsj/Low-resource-language-code/blob/main/ASR_Mitch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Automated voice recognission

In [ ]:
!apt install ffmpeg

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:4.2.7-0ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 13 not upgraded.


In [ ]:
!pip install -U openai-whisper
!pip install sox
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.9/792.9 kB 14.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 64.7 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20230314-py3-none-any.whl size=796910 sha256=26f01b52cfa816f0738bc664ef5d73c8dad5a432cb5411220ddc6a5c182950f6
  Stored in directory: /root/.cache/pip/wheels/b2/13/5f/fe8245f6dc59df505879da4b2129932e342f02a80e6b87f27d
Successfully built openai-whisper
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import whisper
import math
import re
from collections import Counter
import string
from tqdm import tqdm

In [ ]:
#mounting the drive with the dataset
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#initiating wisper and wisper utility functions
model = whisper.load_model("base")

def transcribe(model, filename):
    res = model.transcribe(filename, fp16=False)
    return res["text"]

def transcribeLS(model, filename, languageToken):
    res = model.transcribe(filename, fp16=False, language=languageToken)
    return res["text"]

100%|███████████████████████████████████████| 139M/139M [00:01<00:00, 86.1MiB/s]


In [ ]:
#mitch his scoring stuff
def cosinesimularity(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x]**2 for x in list(vec1.keys())])
    sum2 = sum([vec2[x]**2 for x in list(vec2.keys())])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator

def text_to_vector(text):
    WORD = re.compile(r"\w+")
    words = WORD.findall(text)
    return Counter(words)

def get_cosinesimularity(text1, text2):
    vector1 = text_to_vector(text1.lower().translate(str.maketrans('', '', string.punctuation)))
    vector2 = text_to_vector(text2.lower().translate(str.maketrans('', '', string.punctuation)))

    cosine = cosinesimularity(vector1, vector2)

    return cosine

In [ ]:
LIMIT = 100

In [ ]:
def grab_keywords(sentence, wlen):
    words = sentence.split(" ")
    keywords = []
    for i in words:
        if len(i) >= wlen:
            keywords.append(i)
    return " ".join(keywords)

In [ ]:
#Establish baseline with english test
#loading in the dataset csv
dataEnglish = pd.read_csv("/content/drive/MyDrive/cv-valid-test.csv")
AccList = []
AccListLS = []
KeyWordAcc = []
KeyWordAccLS = []
i = 0

for samp in tqdm(dataEnglish.iterrows()):
  if i == LIMIT:
    break
  file_path = "/content/drive/MyDrive/"+samp[1][0]
  pred = transcribe(model, file_path)
  predLS = transcribeLS(model, file_path, "en")
  text = samp[1][1]
  AccList.append(get_cosinesimularity(pred,text))
  AccListLS.append(get_cosinesimularity(predLS,text))
  KeyWordAcc.append(get_cosinesimularity(grab_keywords(pred ,5), grab_keywords(text, 5)))
  KeyWordAccLS.append(get_cosinesimularity(grab_keywords(predLS ,5), grab_keywords(text, 5)))
  i = i + 1
Average = sum(AccList) / len(AccList)
print("Full list: " + str(AccList))
print("The english average accuracy: " + str(Average))
print("The english keyword spotting accuracy: " + str(sum(KeyWordAcc)/len(KeyWordAcc)))

AverageLS = sum(AccListLS) / len(AccListLS)
print("Full list LS: " + str(AccListLS))
print("The english average accuracy LS: " + str(AverageLS))
print("The english keyword spotting accuracy LS: " + str(sum(KeyWordAccLS)/len(KeyWordAccLS)))

100it [16:32,  9.93s/it]

Full list: [1.0, 0.9999999999999998, 1.0, 0.0, 1.0, 1.0, 0.8333333333333335, 1.0000000000000002, 0.9999999999999999, 1.0, 0.9999999999999999, 0.8999999999999998, 0.9999999999999998, 1.0, 1.0, 0.9999999999999998, 1.0000000000000002, 0.9999999999999998, 0.9999999999999998, 0.8581163303210331, 0.9999999999999999, 0.9999999999999998, 0.9230769230769232, 0.8432740427115678, 1.0, 0.9999999999999999, 0.8333333333333335, 1.0000000000000002, 0.9999999999999998, 1.0000000000000002, 1.0000000000000002, 0.960768922830523, 0.9999999999999999, 1.0, 0.6708203932499369, 1.0, 0.9090909090909091, 1.0, 1.0, 1.0, 0.7378647873726218, 0.9999999999999998, 0.9999999999999999, 1.0, 0.8581163303210331, 1.0000000000000002, 1.0, 1.0, 1.0000000000000002, 0.9999999999999998, 1.0, 0.7499999999999999, 1.0, 1.0, 0.9999999999999998, 1.0, 0.9999999999999998, 1.0000000000000002, 0.9999999999999998, 1.0000000000000002, 1.0, 0.8164965809277261, 0.7171371656006361, 1.0, 1.0000000000000002, 0.8249579113843053, 0.999999999999

In [ ]:
#Establish baseline with Armenian test
#loading in the dataset tsv
dataArmenian = pd.read_csv("/content/drive/MyDrive/CommonVoiceMitch/Armenian/hy-AM/validated.tsv", sep="\t")
AccList = []
AccListLS = []
KeyWordAcc = []
KeyWordAccLS = []
i = 0

for samp in tqdm(dataArmenian.iterrows()):
  if i == LIMIT:
    break
  file_path = "/content/drive/MyDrive/CommonVoiceMitch/Armenian/hy-AM/clips/"+samp[1][1]
  pred = transcribe(model, file_path)
  predLS = transcribeLS(model, file_path, "hy")
  text = samp[1][2]
  AccList.append(get_cosinesimularity(pred,text))
  AccListLS.append(get_cosinesimularity(predLS,text))
  KeyWordAcc.append(get_cosinesimularity(grab_keywords(pred ,5), grab_keywords(text, 5)))
  KeyWordAccLS.append(get_cosinesimularity(grab_keywords(predLS ,5), grab_keywords(text, 5)))
  i = i + 1
Average = sum(AccList) / len(AccList)
print("Full list: " + str(AccList))
print("The armenian average accuracy: " + str(Average))
print("The armenian keyword spotting accuracy: " + str(sum(KeyWordAcc)/len(KeyWordAcc)))

AverageLS = sum(AccListLS) / len(AccListLS)
print("Full list LS: " + str(AccListLS))
print("The armenian average accuracy LS: " + str(AverageLS))
print("The armenian keyword spotting accuracy LS: " + str(sum(KeyWordAccLS)/len(KeyWordAccLS)))

100it [1:59:38, 71.78s/it]

Full list: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
The armenian average accuracy: 0.0
The armenian keyword spotting accuracy: 0.0
Full list LS: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [ ]:
#Establish baseline with Azerbaijani test
#loading in the dataset tsv
dataAzerbaijani = pd.read_csv("/content/drive/MyDrive/CommonVoiceMitch/Azerbaijani/az/validated.tsv", sep="\t")
AccList = []
AccListLS = []
KeyWordAcc = []
KeyWordAccLS = []
i = 0

for samp in tqdm(dataAzerbaijani.iterrows()):
  if i == LIMIT:
    break
  file_path = "/content/drive/MyDrive/CommonVoiceMitch/Azerbaijani/az/clips/"+samp[1][1]
  pred = transcribe(model, file_path)
  predLS = transcribeLS(model, file_path, "az")
  text = samp[1][2]
  AccList.append(get_cosinesimularity(pred,text))
  AccListLS.append(get_cosinesimularity(predLS,text))
  KeyWordAcc.append(get_cosinesimularity(grab_keywords(pred ,5), grab_keywords(text, 5)))
  KeyWordAccLS.append(get_cosinesimularity(grab_keywords(predLS ,5), grab_keywords(text, 5)))
  i = i + 1
Average = sum(AccList) / len(AccList)
print("Full list: " + str(AccList))
print("The azerbaijani average accuracy: " + str(Average))
print("The azerbaijani keyword spotting accuracy: " + str(sum(KeyWordAcc)/len(KeyWordAcc)))

AverageLS = sum(AccListLS) / len(AccListLS)
print("Full list LS: " + str(AccListLS))
print("The azerbaijani average accuracy LS: " + str(AverageLS))
print("The azerbaijani keyword spotting accuracy LS: " + str(sum(KeyWordAccLS)/len(KeyWordAccLS)))

87it [22:54, 15.80s/it]

Full list: [0.25087260300212727, 0.0, 0.08770580193070293, 0.0, 0.07692307692307693, 0.33333333333333337, 0.0, 0.0, 0.09534625892455924, 0.21081851067789195, 0.12499999999999997, 0.0, 0.0, 0.0, 0.33333333333333337, 0.35856858280031806, 0.08006407690254358, 0.38138503569823695, 0.5222329678670935, 0.3086066999241838, 0.0, 0.2857142857142857, 0.25087260300212727, 0.37573457465108967, 0.08451542547285165, 0.09622504486493763, 0.40089186286863654, 0.0, 0.0, 0.33333333333333337, 0.0, 0.16666666666666669, 0.0, 0.15384615384615385, 0.19999999999999996, 0.15384615384615385, 0.22237479499833038, 0.1336306209562122, 0.2307692307692308, 0.13363062095621217, 0.0, 0.13363062095621217, 0.0, 0.0, 0.0, 0.24019223070763074, 0.0, 0.09534625892455924, 0.08058229640253803, 0.0, 0.2307692307692308, 0.2519763153394848, 0.0, 0.14285714285714285, 0.0, 0.0, 0.35355339059327373, 0.4472135954999579, 0.3999999999999999, 0.4803844614152615, 0.2727272727272727, 0.0, 0.0, 0.10341753799900383, 0.0, 0.1054092553389459

In [ ]:
#Establish baseline with Icelandic test
#loading in the dataset tsv
dataIcelandic = pd.read_csv("/content/drive/MyDrive/CommonVoiceMitch/Icelandic/is/other.tsv", sep="\t")
AccList = []
AccListLS = []
KeyWordAcc = []
KeyWordAccLS = []
i = 0

for samp in tqdm(dataIcelandic.iterrows()):
  if i == LIMIT:
    break
  file_path = "/content/drive/MyDrive/CommonVoiceMitch/Icelandic/is/clips/"+samp[1][1]
  pred = transcribe(model, file_path)
  predLS = transcribeLS(model, file_path, "is")
  text = samp[1][2]
  AccList.append(get_cosinesimularity(pred,text))
  AccListLS.append(get_cosinesimularity(predLS,text))
  KeyWordAcc.append(get_cosinesimularity(grab_keywords(pred ,5), grab_keywords(text, 5)))
  KeyWordAccLS.append(get_cosinesimularity(grab_keywords(predLS ,5), grab_keywords(text, 5)))
  i = i + 1
Average = sum(AccList) / len(AccList)
print("Full list: " + str(AccList))
print("The icelandic average accuracy: " + str(Average))
print("The icelandic keyword spotting accuracy: " + str(sum(KeyWordAcc)/len(KeyWordAcc)))

AverageLS = sum(AccListLS) / len(AccListLS)
print("Full list LS: " + str(AccListLS))
print("The icelandic average accuracy LS: " + str(AverageLS))
print("The icelandic keyword spotting accuracy LS: " + str(sum(KeyWordAccLS)/len(KeyWordAccLS)))

10it [07:09, 42.99s/it]

Full list: [0.0, 0.0, 0.0, 0.0, 0.09128709291752768, 0.0, 0.24999999999999994, 0.0, 0.09622504486493763, 0.0]
The icelandic average accuracy: 0.04375121377824652
The icelandic keyword spotting accuracy: 0.015430334996209192
Full list LS: [0.12433397443204185, 0.23570226039551587, 0.0, 0.08703882797784893, 0.08703882797784893, 0.0, 0.13363062095621217, 0.0, 0.08703882797784893, 0.0]
The icelandic average accuracy LS: 0.07547833397173166
The icelandic keyword spotting accuracy LS: 0.0


In [ ]:
#Establish baseline with Kazakh test
#loading in the dataset tsv
dataKazakh = pd.read_csv("/content/drive/MyDrive/CommonVoiceMitch/Kazakh/kk/validated.tsv", sep="\t")
AccList = []
AccListLS = []
KeyWordAcc = []
KeyWordAccLS = []
i = 0

for samp in tqdm(dataKazakh.iterrows()):
  if i == LIMIT:
    break
  file_path = "/content/drive/MyDrive/CommonVoiceMitch/Kazakh/kk/clips/"+samp[1][1]
  pred = transcribe(model, file_path)
  predLS = transcribeLS(model, file_path, "kk")
  text = samp[1][2]
  AccList.append(get_cosinesimularity(pred,text))
  AccListLS.append(get_cosinesimularity(predLS,text))
  KeyWordAcc.append(get_cosinesimularity(grab_keywords(pred ,5), grab_keywords(text, 5)))
  KeyWordAccLS.append(get_cosinesimularity(grab_keywords(predLS ,5), grab_keywords(text, 5)))
  i = i + 1
Average = sum(AccList) / len(AccList)
print("Full list: " + str(AccList))
print("The kazakh average accuracy: " + str(Average))
print("The kazakh keyword spotting accuracy: " + str(sum(KeyWordAcc)/len(KeyWordAcc)))

AverageLS = sum(AccListLS) / len(AccListLS)
print("Full list LS: " + str(AccListLS))
print("The kazakh average accuracy LS: " + str(AverageLS))
print("The kazakh keyword spotting accuracy LS: " + str(sum(KeyWordAccLS)/len(KeyWordAccLS)))

100it [38:17, 22.98s/it]

Full list: [0.0, 0.0, 0.0, 0.15811388300841897, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2857142857142857, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0816496580927726, 0.0, 0.0, 0.14142135623730948, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.09128709291752767, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.20412414523193148, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2357022603955158, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
The kazakh average accuracy: 0.011980126815977616
The kazakh keyword spotting accuracy: 0.0065042749475954405
Full list LS: [0.0, 0.0, 0.6249999999999999, 0.0, 0.0, 0.0, 0.0, 0.1432229748078866, 0.0, 0.0, 0.0, 0.18257418583505536, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2857142857142857, 0.0, 0.0, 0.19999999999999996, 0.35355339059327373, 0.0, 0.0, 0.1178511301977579, 0.0, 0.0, 0

In [ ]:
#Establish baseline with Korean test
#loading in the dataset tsv
dataKorean = pd.read_csv("/content/drive/MyDrive/CommonVoiceMitch/Korean/ko/validated.tsv", sep="\t")
AccList = []
AccListLS = []
KeyWordAcc = []
KeyWordAccLS = []
i = 0

for samp in tqdm(dataKorean.iterrows()):
  if i == LIMIT:
    break
  file_path = "/content/drive/MyDrive/CommonVoiceMitch/Korean/ko/clips/"+samp[1][1]
  pred = transcribe(model, file_path)
  predLS = transcribeLS(model, file_path, "ko")
  text = samp[1][2]
  AccList.append(get_cosinesimularity(pred,text))
  AccListLS.append(get_cosinesimularity(predLS,text))
  KeyWordAcc.append(get_cosinesimularity(grab_keywords(pred ,5), grab_keywords(text, 5)))
  KeyWordAccLS.append(get_cosinesimularity(grab_keywords(predLS ,5), grab_keywords(text, 5)))
  i = i + 1
Average = sum(AccList) / len(AccList)
print("Full list: " + str(AccList))
print("The korean average accuracy: " + str(Average))
print("The korean keyword spotting accuracy: " + str(sum(KeyWordAcc)/len(KeyWordAcc)))

AverageLS = sum(AccListLS) / len(AccListLS)
print("Full list LS: " + str(AccListLS))
print("The korean average accuracy LS: " + str(AverageLS))
print("The korean keyword spotting accuracy LS: " + str(sum(KeyWordAccLS)/len(KeyWordAccLS)))

100it [17:19, 10.39s/it]

Full list: [0.9258200997725514, 0.6943650748294137, 0.6172133998483676, 0.6249999999999999, 0.7142857142857142, 1.0000000000000002, 0.38138503569823695, 0.6666666666666666, 0.4803844614152615, 0.6324555320336759, 0.5999999999999999, 0.9999999999999999, 0.4472135954999579, 0.5892556509887895, 0.5892556509887895, 0.0, 0.5714285714285714, 0.857142857142857, 0.2581988897471611, 0.7833494518006403, 0.7999999999999998, 0.857142857142857, 0.8432740427115678, 0.6708203932499369, 0.37499999999999994, 1.0000000000000002, 0.6324555320336759, 0.3086066999241838, 0.16666666666666669, 1.0, 0.6666666666666667, 0.33806170189140655, 0.6249999999999999, 0.19999999999999996, 0.75, 0.3999999999999999, 0.4714045207910316, 0.6249999999999999, 0.7627700713964739, 0.18181818181818182, 0.6249999999999999, 0.5000000000000001, 0.2721655269759087, 0.13608276348795434, 0.8333333333333335, 0.75, 0.75, 0.0, 0.6172133998483676, 0.5555555555555556, 0.5555555555555556, 1.0000000000000002, 0.0, 0.6363636363636364, 0.583

In [ ]:
#Establish baseline with Macedonian test
#loading in the dataset tsv
dataMacedonian = pd.read_csv("/content/drive/MyDrive/CommonVoiceMitch/Macedonian/mk/validated.tsv", sep="\t")
AccList = []
AccListLS = []
KeyWordAcc = []
KeyWordAccLS = []
i = 0

for samp in tqdm(dataMacedonian.iterrows()):
  if i == LIMIT:
    break
  file_path = "/content/drive/MyDrive/CommonVoiceMitch/Macedonian/mk/clips/"+samp[1][1]
  pred = transcribe(model, file_path)
  predLS = transcribeLS(model, file_path, "mk")
  text = samp[1][2]
  AccList.append(get_cosinesimularity(pred,text))
  AccListLS.append(get_cosinesimularity(predLS,text))
  KeyWordAcc.append(get_cosinesimularity(grab_keywords(pred ,5), grab_keywords(text, 5)))
  KeyWordAccLS.append(get_cosinesimularity(grab_keywords(predLS ,5), grab_keywords(text, 5)))
  i = i + 1
Average = sum(AccList) / len(AccList)
print("Full list: " + str(AccList))
print("The macedonian average accuracy: " + str(Average))
print("The macedonian keyword spotting accuracy: " + str(sum(KeyWordAcc)/len(KeyWordAcc)))

AverageLS = sum(AccListLS) / len(AccListLS)
print("Full list LS: " + str(AccListLS))
print("The macedonian average accuracy LS: " + str(AverageLS))
print("The macedonian keyword spotting accuracy LS: " + str(sum(KeyWordAccLS)/len(KeyWordAccLS)))

100it [21:33, 12.94s/it]

Full list: [0.0, 0.4714045207910316, 0.16116459280507606, 0.0, 0.22360679774997896, 0.0, 0.22613350843332272, 0.6030226891555273, 0.5222329678670935, 0.4285714285714285, 0.4708709557974188, 0.7142857142857142, 0.4714045207910316, 0.0, 0.0, 0.0, 0.23145502494313788, 0.0, 0.0, 0.18181818181818182, 0.08058229640253803, 0.0, 0.6092717958449424, 0.36084391824351614, 0.0, 0.0, 0.0, 0.26111648393354675, 0.0, 0.0, 0.3651483716701107, 0.0, 0.0, 0.3481553119113957, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5270462766947299, 0.0, 0.6324555320336759, 0.48666426339228763, 0.0, 0.17928429140015903, 0.0, 0.10660035817780521, 0.10540925533894598, 0.22360679774997896, 0.0, 0.4629100498862757, 0.0, 0.36514837167011077, 0.0, 0.0, 0.0, 0.26906911759852503, 0.0, 0.0, 0.4714045207910316, 0.0, 0.17541160386140586, 0.0, 0.0, 0.0, 0.0, 0.09128709291752768, 0.13363062095621217, 0.45226701686664544, 0.6249999999999999, 0.1178511301977579, 0.08006407690254358, 0.0, 0.0, 0.0, 0.0, 0.0, 0.4216370213557839, 0.2727272727272727, 0.0

In [ ]:
#Establish baseline with Malay test
#loading in the dataset tsv
dataMalay = pd.read_csv("/content/drive/MyDrive/CommonVoiceMitch/Malay/ml/validated.tsv", sep="\t")
AccList = []
AccListLS = []
KeyWordAcc = []
KeyWordAccLS = []
i = 0

for samp in tqdm(dataMalay.iterrows()):
  if i == LIMIT:
    break
  file_path = "/content/drive/MyDrive/CommonVoiceMitch/Malay/ml/clips/"+samp[1][1]
  pred = transcribe(model, file_path)
  predLS = transcribeLS(model, file_path, "ml")
  text = samp[1][2]
  AccList.append(get_cosinesimularity(pred,text))
  AccListLS.append(get_cosinesimularity(predLS,text))
  KeyWordAcc.append(get_cosinesimularity(grab_keywords(pred ,5), grab_keywords(text, 5)))
  KeyWordAccLS.append(get_cosinesimularity(grab_keywords(predLS ,5), grab_keywords(text, 5)))
  i = i + 1
Average = sum(AccList) / len(AccList)
print("Full list: " + str(AccList))
print("The malay average accuracy: " + str(Average))
print("The malay keyword spotting accuracy: " + str(sum(KeyWordAcc)/len(KeyWordAcc)))

AverageLS = sum(AccListLS) / len(AccListLS)
print("Full list LS: " + str(AccListLS))
print("The malay average accuracy LS: " + str(AverageLS))
print("The malay keyword spotting accuracy LS: " + str(sum(KeyWordAccLS)/len(KeyWordAccLS)))

100it [45:15, 27.15s/it]

Full list: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
The malay average accuracy: 0.0
The malay keyword spotting accuracy: 0.0
Full list LS: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

In [ ]:
#Establish baseline with Nepali test
#loading in the dataset tsv
dataNepali = pd.read_csv("/content/drive/MyDrive/CommonVoiceMitch/Nepali/ne-NP/validated.tsv", sep="\t")
AccList = []
AccListLS = []
KeyWordAcc = []
KeyWordAccLS = []
i = 0

for samp in tqdm(dataNepali.iterrows()):
  if i == LIMIT:
    break
  file_path = "/content/drive/MyDrive/CommonVoiceMitch/Nepali/ne-NP/clips/"+samp[1][1]
  pred = transcribe(model, file_path)
  predLS = transcribeLS(model, file_path, "ne")
  text = samp[1][2]
  AccList.append(get_cosinesimularity(pred,text))
  AccListLS.append(get_cosinesimularity(predLS,text))
  KeyWordAcc.append(get_cosinesimularity(grab_keywords(pred ,5), grab_keywords(text, 5)))
  KeyWordAccLS.append(get_cosinesimularity(grab_keywords(predLS ,5), grab_keywords(text, 5)))
  i = i + 1
Average = sum(AccList) / len(AccList)
print("Full list: " + str(AccList))
print("The nepali average accuracy: " + str(Average))
print("The nepali keyword spotting accuracy: " + str(sum(KeyWordAcc)/len(KeyWordAcc)))

AverageLS = sum(AccListLS) / len(AccListLS)
print("Full list LS: " + str(AccListLS))
print("The nepali average accuracy LS: " + str(AverageLS))
print("The nepali keyword spotting accuracy LS: " + str(sum(KeyWordAccLS)/len(KeyWordAccLS)))

100it [33:12, 19.92s/it]

Full list: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
The nepali average accuracy: 0.0
The nepali keyword spotting accuracy: 0.0
Full list LS: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

In [ ]:
#Establish baseline with Norwegian test
#loading in the dataset tsv
dataNorwegian = pd.read_csv("/content/drive/MyDrive/CommonVoiceMitch/Norwegian/nn-NO/validated.tsv", sep="\t")
AccList = []
AccListLS = []
KeyWordAcc = []
KeyWordAccLS = []
i = 0

for samp in tqdm(dataNorwegian.iterrows()):
  if i == LIMIT:
    break
  file_path = "/content/drive/MyDrive/CommonVoiceMitch/Norwegian/nn-NO/clips/"+samp[1][1]
  pred = transcribe(model, file_path)
  predLS = transcribeLS(model, file_path, "no")
  text = samp[1][2]
  AccList.append(get_cosinesimularity(pred,text))
  AccListLS.append(get_cosinesimularity(predLS,text))
  KeyWordAcc.append(get_cosinesimularity(grab_keywords(pred ,5), grab_keywords(text, 5)))
  KeyWordAccLS.append(get_cosinesimularity(grab_keywords(predLS ,5), grab_keywords(text, 5)))
  i = i + 1
Average = sum(AccList) / len(AccList)
print("Full list: " + str(AccList))
print("The norwegian average accuracy: " + str(Average))
print("The norwegian keyword spotting accuracy: " + str(sum(KeyWordAcc)/len(KeyWordAcc)))

AverageLS = sum(AccListLS) / len(AccListLS)
print("Full list LS: " + str(AccListLS))
print("The norwegian average accuracy LS: " + str(AverageLS))
print("The norwegian keyword spotting accuracy LS: " + str(sum(KeyWordAccLS)/len(KeyWordAccLS)))

100it [23:03, 13.84s/it]

Full list: [0.0, 0.0, 0.06350006350009525, 0.16116459280507606, 0.39223227027636803, 0.28603877677367767, 0.0, 0.3227486121839514, 0.629940788348712, 0.0, 0.22360679774997896, 0.30151134457776363, 0.0, 0.0, 0.4036036763977875, 0.75, 0.0, 0.29999999999999993, 0.4811252243246881, 0.0, 0.0, 0.0, 0.3768891807222045, 0.0, 0.0, 0.33407655239053047, 0.0, 0.9999999999999998, 0.6666666666666667, 0.0, 0.2357022603955158, 0.0, 0.0, 0.0, 0.0, 0.06900655593423542, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.15811388300841897, 0.0, 0.11180339887498948, 0.0, 0.10540925533894598, 0.3442651863295481, 0.14142135623730948, 0.5017452060042545, 0.4714045207910316, 0.1025978352085154, 0.21081851067789195, 0.28867513459481287, 0.4216370213557839, 0.0, 0.11396057645963795, 0.45643546458763845, 0.13801311186847084, 0.7999999999999998, 0.35007002100700246, 0.5477225575051661, 0.30618621784789724, 0.09999999999999998, 0.5809475019311126, 0.12403473458920847, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.35355339059327

In [ ]:
#Establish baseline with Serbian test
#loading in the dataset tsv
dataSerbian = pd.read_csv("/content/drive/MyDrive/CommonVoiceMitch/Serbian/sr/validated.tsv", sep="\t")
AccList = []
AccListLS = []
KeyWordAcc = []
KeyWordAccLS = []
i = 0

for samp in tqdm(dataSerbian.iterrows()):
  if i == LIMIT:
    break
  file_path = "/content/drive/MyDrive/CommonVoiceMitch/Serbian/sr/clips/"+samp[1][1]
  pred = transcribe(model, file_path)
  predLS = transcribeLS(model, file_path, "sr")
  text = samp[1][2]
  AccList.append(get_cosinesimularity(pred,text))
  AccListLS.append(get_cosinesimularity(predLS,text))
  KeyWordAcc.append(get_cosinesimularity(grab_keywords(pred ,5), grab_keywords(text, 5)))
  KeyWordAccLS.append(get_cosinesimularity(grab_keywords(predLS ,5), grab_keywords(text, 5)))
  i = i + 1
Average = sum(AccList) / len(AccList)
print("Full list: " + str(AccList))
print("The serbian average accuracy: " + str(Average))
print("The serbian keyword spotting accuracy: " + str(sum(KeyWordAcc)/len(KeyWordAcc)))

AverageLS = sum(AccListLS) / len(AccListLS)
print("Full list LS: " + str(AccListLS))
print("The serbian average accuracy LS: " + str(AverageLS))
print("The serbian keyword spotting accuracy LS: " + str(sum(KeyWordAccLS)/len(KeyWordAccLS)))

100it [22:09, 13.30s/it]

Full list: [0.0, 0.0, 0.4714045207910316, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.19999999999999996, 0.0, 0.0, 0.0, 0.0, 0.40824829046386296, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.40824829046386296, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2886751345948129, 0.0, 0.0, 0.33333333333333337, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2886751345948129, 0.33333333333333337, 0.33333333333333337, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5773502691896258, 0.0, 0.0, 0.0, 0.0, 0.6324555320336759, 0.0]
The serbian average accuracy: 0.042750571721316855
The serbian keyword spotting accuracy: 0.015477225575051662
Full list LS: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0